# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


# loading data

In [2]:
dataset = pd.read_csv('./CarPrice_Assignment.csv')

In [3]:
dataset.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
dataset.drop(columns=['car_ID'], inplace=True)

# checking for missing values

In [5]:
dataset.isnull().values.any()

False

# shuffling the dataset

In [6]:
from sklearn.utils import shuffle
dataset = shuffle(dataset)
dataset.reset_index(inplace=True, drop=True)

In [7]:
dataset

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,-1,toyota corolla,gas,std,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2326,ohc,four,122,mpfi,3.31,3.54,8.70,92,4200,29,34,8948.0
1,0,mazda glc custom l,gas,std,four,sedan,fwd,front,98.8,177.8,66.5,55.5,2410,ohc,four,122,2bbl,3.39,3.39,8.60,84,4800,26,32,8495.0
2,-1,volvo 245,gas,std,four,wagon,rwd,front,104.3,188.8,67.2,57.5,3042,ohc,four,141,mpfi,3.78,3.15,9.50,114,5400,24,28,16515.0
3,-1,volvo 144ea,gas,std,four,wagon,rwd,front,104.3,188.8,67.2,57.5,3034,ohc,four,141,mpfi,3.78,3.15,9.50,114,5400,23,28,13415.0
4,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.80,134,5500,18,23,21485.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2,audi fox,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.50,110,5500,19,25,15250.0
201,1,dodge rampage,gas,std,two,hatchback,fwd,front,93.7,157.3,63.8,50.8,1876,ohc,four,90,2bbl,2.97,3.23,9.41,68,5500,37,41,5572.0
202,0,honda civic 1500 gl,gas,std,four,wagon,fwd,front,96.5,157.1,63.9,58.3,2024,ohc,four,92,1bbl,2.92,3.41,9.20,76,6000,30,34,7295.0
203,3,saab 99gle,gas,turbo,two,hatchback,fwd,front,99.1,186.6,66.5,56.1,2808,dohc,four,121,mpfi,3.54,3.07,9.00,160,5500,19,26,18150.0


In [8]:
dataset.select_dtypes(include=['object']).dtypes

CarName           object
fueltype          object
aspiration        object
doornumber        object
carbody           object
drivewheel        object
enginelocation    object
enginetype        object
cylindernumber    object
fuelsystem        object
dtype: object

# one hot encoding categorical features

In [9]:
udataset =pd.get_dummies(dataset, columns=['symboling','CarName','fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem'])

In [10]:
udataset.describe()

,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,symboling_-2,symboling_-1,symboling_0,symboling_1,symboling_2,symboling_3,CarName_Nissan versa,CarName_alfa-romero Quadrifoglio,CarName_alfa-romero giulia,CarName_alfa-romero stelvio,CarName_audi 100 ls,CarName_audi 100ls,CarName_audi 4000,CarName_audi 5000,CarName_audi 5000s (diesel),CarName_audi fox,CarName_bmw 320i,CarName_bmw x1,CarName_bmw x3,CarName_bmw x4,CarName_bmw x5,CarName_bmw z4,CarName_buick century,CarName_buick century luxus (sw),CarName_buick century special,CarName_buick electra 225 custom,...,CarName_vw dasher,CarName_vw rabbit,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginelocation_rear,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_eight,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,...,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571,0.014634,0.107317,0.326829,0.263415,0.156098,0.131707,0.004878,0.004878,0.004878,0.004878,0.004878,0.009756,0.004878,0.004878,0.004878,0.004878,0.009756,0.004878,0.009756,0.004878,0.004878,0.004878,0.004878,0.004878,0.004878,0.004878,...,0.004878,0.004878,0.097561,0.902439,0.819512,0.180488,0.560976,0.439024,0.029268,0.039024,0.341463,0.468293,0.121951,0.043902,0.585366,0.370732,0.985366,0.014634,0.058537,0.004878,0.058537,0.721951,0.073171,0.063415,0.019512,0.024390,0.053659,0.775610,0.117073,0.004878,0.004878,0.019512,0.053659,0.321951,0.014634,0.097561,0.004878,0.458537,0.043902,0.004878
std,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332,0.120377,0.310274,0.470202,0.441564,0.363836,0.339000,0.069843,0.069843,0.069843,0.069843,0.069843,0.098531,0.069843,0.069843,0.069843,0.069843,0.098531,0.069843,0.098531,0.069843,0.069843,0.069843,0.069843,0.069843,0.069843,0.069843,...,0.069843,0.069843,0.297446,0.297446,0.385535,0.385535,0.497483,0.497483,0.168970,0.194127,0.475361,0.500215,0.328031,0.205380,0.493865,0.484183,0.120377,0.120377,0.235330,0.069843,0.235330,0.449134,0.261054,0.244304,0.138655,0.154635,0.225894,0.418201,0.322294,0.069843,0.069843,0.138655,0.225894,0.468368,0.120377,0.297446,0.069843,0.499498,0.205380,0.069843
min,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

# Splitting training and testing dataset which is already shuffled

In [11]:
# training and testing datasets

train_dataset=udataset.loc[0:163,:]
test_dataset = udataset.loc[164:,:]

# training data and labels
x_train = train_dataset.drop(columns='price')
y_train = train_dataset.loc[0:163, 'price']

# testing data and labels
x_test = test_dataset.drop(columns=['price'])
y_test = test_dataset.loc[164:,'price']
 


# Normalizing the data

In [12]:
# taking the mean and std of data

mean_data = x_train.iloc[:,0:13].mean(axis=0)
std_data = x_train.iloc[:,0:13].std(axis=0)

# applying mean and std on training data

x_train.iloc[:,0:13] -= mean_data
x_train.iloc[:,0:13] /= std_data

# applying mean and std on testing data
x_test.iloc[:,0:13] -= mean_data
x_test.iloc[:,0:13] /= std_data


# taking the mean and std of labels
mean_labels = y_train.mean(axis=0)
std_labels  = y_train.std(axis=0)

# applying mean and std on training and testing labels
y_train -= mean_labels
y_train /= std_labels

y_test -= mean_labels
y_test /= std_labels



# converting into np arrays

In [13]:
train_data = np.array(x_train.iloc[:])
test_data = np.array(x_test.iloc[:])

train_labels = np.array(y_train.astype('float32'))
test_labels = np.array(y_test.astype('float32'))

# Building and compiling the model

In [14]:
from keras import models
from keras import layers
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(8, activation='relu'))
  model.add(layers.Dense(6, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [15]:
build_model()

# K-fold validation

In [16]:
import numpy as np
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate(
  [train_data[:i * num_val_samples],
  train_data[(i + 1) * num_val_samples:]],
  axis=0)
  partial_train_labels = np.concatenate(
  [train_labels[:i * num_val_samples],
  train_labels[(i + 1) * num_val_samples:]],
  axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_labels,
  epochs=num_epochs, batch_size=1, verbose=1)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
Epoch 1/100
123/123 [==============================] - 1s 1ms/step - loss: 0.9752 - mae: 0.6463
Epoch 2/100
123/123 [==============================] - 0s 1ms/step - loss: 0.6192 - mae: 0.4313
Epoch 3/100
123/123 [==============================] - 0s 1ms/step - loss: 0.7735 - mae: 0.4579
Epoch 4/100
123/123 [==============================] - 0s 1ms/step - loss: 0.5720 - mae: 0.4211
Epoch 5/100
123/123 [==============================] - 0s 1ms/step - loss: 0.5381 - mae: 0.3701
Epoch 6/100
123/123 [==============================] - 0s 1ms/step - loss: 0.3408 - mae: 0.3253
Epoch 7/100
123/123 [==============================] - 0s 1ms/step - loss: 0.3007 - mae: 0.2990
Epoch 8/100
123/123 [==============================] - 0s 1ms/step - loss: 0.1249 - mae: 0.2121
Epoch 9/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0578 - mae: 0.1445
Epoch 10/100
123/123 [==============================] - 0s 1ms/step - loss: 0.0735 - mae: 0.1598
Epoch 11/100
123/12

In [17]:
all_scores

[0.23618298768997192,
 0.22866705060005188,
 0.1836264729499817,
 0.19993856549263]

# running and evaluating the model

In [18]:
model = build_model()
model.fit(train_data, train_labels,
epochs=100, batch_size=16, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_labels)

Epoch 1/100
11/11 [==============================] - 1s 2ms/step - loss: 0.7045 - mae: 0.6633
Epoch 2/100
11/11 [==============================] - 0s 2ms/step - loss: 0.4085 - mae: 0.5125
Epoch 3/100
11/11 [==============================] - 0s 2ms/step - loss: 0.2701 - mae: 0.3644
Epoch 4/100
11/11 [==============================] - 0s 2ms/step - loss: 0.1831 - mae: 0.2925
Epoch 5/100
11/11 [==============================] - 0s 2ms/step - loss: 0.1699 - mae: 0.2711
Epoch 6/100
11/11 [==============================] - 0s 2ms/step - loss: 0.1052 - mae: 0.2079
Epoch 7/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0674 - mae: 0.1724
Epoch 8/100
11/11 [==============================] - 0s 3ms/step - loss: 0.0577 - mae: 0.1669
Epoch 9/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0530 - mae: 0.1533
Epoch 10/100
11/11 [==============================] - 0s 2ms/step - loss: 0.0493 - mae: 0.1464
Epoch 11/100
11/11 [==============================] - 0s 2m

In [19]:
test_mae_score

0.14847978949546814

In [20]:
test_labels[0]

0.35393673

# prediction

In [21]:
prediction = model.predict(test_data[0].reshape(1, 204))

In [23]:
prediction

array([[0.4772317]], dtype=float32)